In [9]:
import sys 
sys.path.append('..')
from src.imagebind import get_image_features, define_model, feature_dim
from src.build_classifier import get_classifier
from src.train_clf import train

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as FT
import torch

In [3]:
sys.path.append('..')
from cifar.cifarRawCorrupted import get_original_loaders, get_corrupt_loaders

In [4]:
device = 'cuda:0'
model = define_model(device=device)

In [5]:
c_loader = get_corrupt_loaders(model_name='imagebind', severity=1)


In [11]:
imagebind_clf = get_classifier(feature_dim, output_classes=10, n_layers=1).to(device)
train_loader, val_loader, test_loader = get_original_loaders(batch_size=1024, model_name='blip') 
test_corrupt_loader = get_corrupt_loaders(batch_size=1024, model_name='blip')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(imagebind_clf.parameters(), lr=0.001)
n_epochs = 2

In [13]:
losses, accs, val_losses, val_accs = train(model, imagebind_clf, optim=optim, loss_fn=loss_fn,
                                           train_loader=train_loader, val_loader=val_loader,
                                           feature_fn=get_image_features, epochs=n_epochs, device=device) #TODO resize im in clip transforms

/home/omoussa/miniconda3/envs/brn/lib/python3.9/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


initial loss 2.3026005744934084 and initial accuracy 0.11320750415325165
 train loss: 2.017957925796509, val loss: 1.585170841217041, Train accuracy 0.8608154058456421, val accuracy 0.9844068288803101 
 train loss: 1.5098349273204803, val loss: 1.4875984907150268, Train accuracy 0.9857177734375, val accuracy 0.9866749048233032 


In [15]:
torch.save(imagebind_clf.state_dict(), '../saved_models/imagebind_clf.pth')


In [17]:
def get_acc(gt, preds = None):
    if preds is not None: 
        return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
        
    
    return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
    

def get_test_acc(emb_model, model, test_loader, feature_fn, device='cuda'):
    eval_acc = []
    eval_losses = []
    for eval_batch in test_loader:
        if len(eval_batch)>2:
            _, ims, labels = eval_batch
        else: 
            ims, labels = eval_batch
        ims, labels = ims.to(device), labels.to(device)
        with torch.no_grad():
            features = feature_fn(emb_model, ims).squeeze()
            preds = model(features)
            val_acc = get_acc(labels.view(-1,), preds)
        
        eval_acc.append(val_acc)
    
    return np.mean(eval_acc)
            # 
test_acc_orig = racc =  get_test_acc(model, imagebind_clf, test_loader, get_image_features, device=device,)
 
print(test_acc_orig)

0.986067


In [18]:
corrupts_dict = {}
corrupt_g_acc = []
for cr in ['gaussian_noise', 'speckle_noise', 'impulse_noise', 'shot_noise', ]:
    corrupts_dict[cr] = {}
    for sev in [1, 2, 3, 4, 5]:
        test_loader_corrupt = get_corrupt_loaders(batch_size=1024, corruption_type=cr, severity=sev, model_name='blip')
        acc =  get_test_acc(model, imagebind_clf, test_loader_corrupt, get_image_features, device=device,)
                                

        corrupts_dict[cr][sev]=acc

In [19]:
corrupts_dict

{'gaussian_noise': {1: 0.9197146,
  2: 0.82722014,
  3: 0.70214444,
  4: 0.6376993,
  5: 0.58301777},
 'speckle_noise': {1: 0.955114,
  2: 0.8844547,
  3: 0.8393275,
  4: 0.7380939,
  5: 0.63478357},
 'impulse_noise': {1: 0.9696628,
  2: 0.94978875,
  3: 0.92277783,
  4: 0.8465541,
  5: 0.757954},
 'shot_noise': {1: 0.95297945,
  2: 0.9128707,
  3: 0.7808095,
  4: 0.7160694,
  5: 0.6023896}}